In [42]:
import json
from tqdm import tqdm
from src.eval import evaluate_on_coco_caption
import evaluate

audio_json = '/pasteur/u/yuhuiz/iccv_c3/cache/generation_data_audio_clotho_imagebind.pkl_c3_0.1_1.json'

with open(audio_json, 'rb') as f:
    gens, refs = json.load(f)

gens, refs = gens[0], refs[0]
gens = [gen.replace("<|endoftext|>", "").strip() for gen in gens]

pred_file = 'audio_pred_coco_format.json'
label_file = 'audio_label_coco_format.json'
out_file = 'audio_metrics.json'

print(len(gens), len(refs))

1045 1045


In [34]:
def reformat_captions_list_to_coco(captions, out_path, label=False):

    labels = {"annotations": [], "images": []}
    
    for id, caption in tqdm(enumerate(captions)):
        image_dict = {"id": id}
        labels["images"].append(image_dict)

        caption_dict = {"image_id": id, "caption": caption, "id": id}
        labels["annotations"].append(caption_dict)
        
    if not label:
        labels = labels['annotations']
        print(f"Total number of predictions: {len(labels)}")
    else:
        print(f"Total number of annotations: {len(labels['annotations'])}")
        print(f"Total number of audio clips: {len(labels['images'])}")
    with open(out_path, 'w') as f:
        json.dump(labels, f)
        


In [40]:
with open(pred_file, 'r') as f:
    preds = json.load(f)

with open(label_file, 'r') as f:
    labels = json.load(f)

print(preds[:10])
print("="*80)
print(labels['annotations'][:10])

[{'image_id': 0, 'caption': 'A vehicle engine is running and idling while a vehicle is moving along the road.', 'id': 0}, {'image_id': 1, 'caption': 'A train horn blaring and a train horn blaring in the background.', 'id': 1}, {'image_id': 2, 'caption': 'A vehicle is driving down the road with a lot of traffic in the background.', 'id': 2}, {'image_id': 3, 'caption': 'A person is walking through a room with a wooden floor creaking.', 'id': 3}, {'image_id': 4, 'caption': 'A person is walking through a room with a wooden floor creaking.', 'id': 4}, {'image_id': 5, 'caption': 'A group of people are talking to each other and a bell rings three times.', 'id': 5}, {'image_id': 6, 'caption': 'A group of people are talking to each other and a bell rings three times.', 'id': 6}, {'image_id': 7, 'caption': 'A group of people are talking to each other and a bell rings three times.', 'id': 7}, {'image_id': 8, 'caption': 'A group of people are talking to each other and a bell rings three times.', '

In [35]:
reformat_captions_list_to_coco(gens, pred_file, label=False)
reformat_captions_list_to_coco(refs, label_file, label=True)

1045it [00:00, 241450.32it/s]


Total number of predictions: 1045


1045it [00:00, 585030.39it/s]

Total number of annotations: 1045
Total number of audio clips: 1045


In [36]:
metrics_dict = evaluate_on_coco_caption(pred_file, label_file, out_file)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
tokenization...


PTBTokenizer tokenized 14169 tokens at 77997.45 tokens per second.
PTBTokenizer tokenized 16608 tokens at 105098.08 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 14519, 'reflen': 12009, 'guess': [14519, 13474, 12429, 11384], 'correct': [3023, 521, 161, 46]}
ratio: 1.2090099092346398
Bleu_1: 0.208
Bleu_2: 0.090
Bleu_3: 0.047
Bleu_4: 0.025
computing METEOR score...
METEOR: 0.087
computing Rouge score...
ROUGE_L: 0.213
computing CIDEr score...
CIDEr: 0.177
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.5 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [2.6 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.4

SPICE evaluation took: 52.69 s
SPICE: 0.071


In [41]:
print(metrics_dict)

{'Bleu_1': 0.20820993181347144, 'Bleu_2': 0.08972662060180385, 'Bleu_3': 0.047069989714806325, 'Bleu_4': 0.02547849751162802, 'METEOR': 0.08673830974499665, 'ROUGE_L': 0.21289196379963496, 'CIDEr': 0.17664520724618576, 'SPICE': 0.0709709072601069}


In [43]:
rouge = evaluate.load('rouge')
rouge_metrics = rouge.compute(predictions=gens, references=refs)
print(rouge_metrics)

meteor = evaluate.load('meteor')
meteor_metrics = meteor.compute(predictions=gens, references=refs)
print(meteor_metrics)

TypeError: compute() takes 1 positional argument but 3 were given